In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
i=0
curr = ''
with open('quartileData\\taskData_barrattBottom.csv', 'r' ) as theFile:
    reader = csv.reader(theFile)
    headers = next(reader, None)
    for line in reader:
        if line[70] != curr:
            if i > 0:
                block = [
                    {
                        'action': np.array([actions])-1,
                        'state': np.array([states]),
#                         'reward': np.array([rewards]),
                        'id': curr,

                        'block': 0
                    }

                ]
#                 if i<226:
                data[curr] = block
#                 else:
                test_data[curr] = block
            curr = line[70]
            i += 1
            actions = []
            states = []
            rewards = []
        actions.append(int(float(line[8])))
        states.append([int(float(line[3])),int(float(line[4]))])
        rewards.append(int(float(line[10])))

In [3]:
data

{'A10KUKZO8X00FX': [{'action': array([[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
           0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
           0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
           0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
           0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
           0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0]]),
   'state': array([[[ 7, 11],
           [ 3, 11],
           [ 7, 11],
           [ 7,  3],
           [ 7, 11],
           [ 3, 11],
           [11,  3],
           [ 3,  7],
           [ 7, 11],
           [ 7,  3],
           [11,  3],
           [11,  7],
           [ 3,  7],
           [ 7, 11],
           [11,  3],
           [ 7,  3],
           [ 7,  1],
           [ 7,  1],
           [ 1,  7],
           [ 1,  7],
 

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn_noFeatures.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn_noFeatures.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/taskData_barrattBottom_noReward/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-08-12 10:26:41,743 - DeepRL - DEBUG - version control: (None, None)
2021-08-12 10:26:41,744 - DeepRL - DEBUG - learning rate: 0.01
2021-08-12 10:26:41,746 - DeepRL - DEBUG - global iters: 50
2021-08-12 10:26:41,747 - DeepRL - DEBUG - training data-points: 278
2021-08-12 10:26:41,748 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-08-12 10:26:43,294 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/taskData_barrattBottom_noReward/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


TypeError: beh_feed() missing 1 required positional argument: 'states'

In [ ]:
# from actionflow.data.data_process import DataProcess
# train_merged = DataProcess.merge_data(data)
# train_merged

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [ ]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

In [ ]:
# test_data["S248"]